<a href="https://colab.research.google.com/github/kib57987/Python/blob/main/stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [90]:
# prompt: get data from https://mops.twse.com.tw/mops/web/ajax_t163sb04 with  POST method. parameter is a json object
# {'encodeURIComponent':1,
#         'step':1,
#         'firstin':1,
#         'off':1,
#         'TYPEK':'sii',
#         'year':112,
#         'season':1}
import requests
import pandas as pd
import re
import numpy as np

category_bank = 1   #銀行業
category_security = 2 #證券業
category_general = 3  #一般業
category_bank_holding = 4  #金控業
category_insurance = 5  #保險業
category_other = 6  #其他

def get_eps(year, season):
    '''
    Get EPS

    Parameters:
        year  (int):
        season (int): 1 to 4

    Returns:
        NA

    Example:
        NA
    '''
    url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb04'

    data = {
        'encodeURIComponent':1,
        'step':1,
        'firstin':1,
        'off':1,
        'TYPEK':'sii',
        'year':year,
        'season':season
    }

    response = requests.post(url, data=data)

    if response.status_code == 200:
        try:
            dfs = pd.read_html(response.text, header=None)
            # 銀行業
            cleaned_df = dfs[category_bank].iloc[0:, [0, 1, 21]].applymap(str)
            print(cleaned_df.to_string())
            #證券業
            cleaned_df = dfs[category_security].iloc[0:, [0, 1, 21]].applymap(str)
            print(cleaned_df.to_string())
            #一般業
            cleaned_df = dfs[category_general].iloc[0:, [0, 1, 29]].applymap(str)
            print(cleaned_df.to_string())
            #金控業
            cleaned_df = dfs[category_bank_holding].iloc[0:, [0, 1, 21]].applymap(str)
            print(cleaned_df.to_string())
            #保險業
            cleaned_df = dfs[category_insurance].iloc[0:, [0, 1, 22]].applymap(str)
            print(cleaned_df.to_string())
            #其他
            cleaned_df = dfs[category_other].iloc[0:, [0, 1, 17]].applymap(str)
            print(cleaned_df.to_string())
            #cleaned_df.to_csv('output.csv', index=False, encoding='utf-8')  # 将清理后的数据写入 CSV 文件
        except ValueError as e:
            print("ValueError occurred while parsing HTML tables:", e)
    else:
        print('Error occurred:', response.status_code)



def get_eps_by_company_id(year, season, company_id):
    '''
    Get EPS data by company ID

    Parameters:
        year (int): The year for the EPS data
        season (int): The season for the EPS data (1 to 4)
        company_id (str): The ID of the company

    Returns:
        pd.DataFrame: A DataFrame containing EPS data for the specified company

    Example:
        get_eps_by_company_id(2023, 1, '2801')
    '''
    url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb04'

    data = {
        'encodeURIComponent': 1,
        'step': 1,
        'firstin': 1,
        'off': 1,
        'TYPEK': 'sii',
        'year': year,
        'season': season
    }

    response = requests.post(url, data=data)
    eps_data = {}

    if response.status_code == 200:
        dfs = pd.read_html(response.text, header=None)

        # 在所有dfs中搜索给定的公司ID
        for df in dfs[1:]:
            if not df.empty and company_id in df.iloc[:, 0].values:
                # 如果找到了公司ID，则提取相应的数据并返回 EPS 值
                for category in range(1, 7):
                    if category == 1:
                        eps_column = 21
                    elif category == 2:
                        eps_column = 21
                    elif category == 3:
                        eps_column = 29
                    elif category == 4:
                        eps_column = 21
                    elif category == 5:
                        eps_column = 22
                    elif category == 6:
                        eps_column = 17
                    try:
                        eps_value = float(df[df.iloc[:, 0] == company_id].iloc[:, eps_column].astype(str).values[0])
                        return eps_value
                    except (ValueError, IndexError):
                        continue

        # 如果未找到公司ID，则打印错误消息
        print("Company ID not found in the available data.")
    else:
        print('Error occurred:', response.status_code)

# Example usage:
year = 112
season = 1
company_id = 2603
print(get_eps_by_company_id(year, season, company_id))
#list_eps(118, 1)


2.38
